#### Оптимизация гиперпараметров

Поиск оптимального количества нейронов в слоях

In [ ]:
from keras_tuner.tuners import BayesianOptimization
import numpy as np
from sklearn.model_selection import train_test_split

from config import DATA_DIR

In [ ]:
pairs = np.load(f'{DATA_DIR}/pairs.npy', mmap_mode='r+')
labels = np.load(f'{DATA_DIR}/labels.npy', mmap_mode='r+')

In [ ]:
pairs.shape, labels.shape

In [ ]:
X_train_pairs, X_test_pairs, y_train_pairs, y_test_pairs = train_test_split(pairs, labels, test_size=0.1, random_state=42)

X_train_pairs.shape, y_train_pairs.shape, X_test_pairs.shape, y_test_pairs.shape

Функция создания нейронной сети с изменением гиперпараметров

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

def get_cnn_block(depth):
    return Sequential([Conv2D(depth, 3, 1),
                       BatchNormalization(),
                       ReLU()])

def build_model(hp):

    IS = 128

    img_a_inp = Input((IS, IS), name='img_a_inp')
    img_b_inp = Input((IS, IS), name='img_b_inp')

    DEPTH = 64
    cnn = Sequential([Reshape((IS, IS, 1)),
                      get_cnn_block(DEPTH),
                      get_cnn_block(DEPTH * 2),
                      GlobalAveragePooling2D(),
                      Dropout(0.2),
                      Flatten(),
                      Dense(units=hp.Int('dense1_units_input',
                                         min_value=64,
                                         max_value=512,
                                         step=32),
                            activation='relu')])

    feature_vector_A = cnn(img_a_inp)
    feature_vector_B = cnn(img_b_inp)

    concat = Concatenate()([feature_vector_A, feature_vector_B])

    dense = Dense(units=hp.Int('dense2_units_input',
                               min_value=64,
                               max_value=512,
                               step=32),
                  activation='relu')(concat)
    dropout = Dropout(0.5)(dense)
    output = Dense(1, activation='sigmoid')(dropout)

    model = Model(inputs=[img_a_inp, img_b_inp], outputs=output)

    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])
    return model

Настройки тюнера

In [ ]:
tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # нужно несколько сотен
    directory='test_directory')

Пространство поиска

In [ ]:
tuner.search_space_summary()

Подбор гиперпараметров

In [ ]:
tuner.search(x=[X_train_pairs[:, 0, :, :], X_train_pairs[:, 1, :, :]],
             y=y_train_pairs,
             batch_size=36,
             epochs=5,
             validation_split=0.2,
             verbose=1)

In [ ]:
tuner.results_summary()

Получаем три лучших модели

In [ ]:
models = tuner.get_best_models(num_models=3)

In [ ]:
for model in models:
    model.summary()